In [1]:
import json

REQUEST = json.dumps({ # Объявляем глобальную переменную, в которую будут записаны парадаваемые параметры PUT и GET
'path' : {},  # Список параметров, переданных в адресной строке
'args' : {}   # Список PUT-параметров
})

In [2]:
import sqlite3 # Загружаем библиотеку
 
conn = sqlite3.connect("McScv.sqlite") # Создаем соединение с базой данных (файлом с именем McScv.sqlite)

cursor = conn.cursor()  # Создаем курсор
# Если не существует таблицы foods - создаем ее
cursor.execute("create table if not exists foods (id INTEGER PRIMARY KEY, name VARCHAR(255), price INTEGER, calories INTEGER, deleted INTEGER NOT NULL DEFAULT 0)")

cursor.close() # Закрываем курсор

In [7]:
def foodGetList():
    conn.row_factory = sqlite3.Row # Выбираем режим, при котором можно обращаться к полям запроса по именам
    cursor = conn.cursor()  # Создаем курсор
    cursor.execute("SELECT * FROM foods WHERE deleted = 0")
    results = cursor.fetchall() # Помещаем результаты запроса в переменную
    cursor.close() # Закрываем курсор

    fl = [] # Переменная - массив с результатами
    for r in results:
       fl.append({"id" : r["id"], "name" : r["name"], "price" : r["price"], "calories" : r["calories"]}) 
    return fl
    
def foodInfo(id):
    conn.row_factory = sqlite3.Row # Выбираем режим, при котором можно обращаться к полям запроса по именам
    cursor = conn.cursor()  # Создаем курсор
    cursor.execute("SELECT * FROM foods WHERE id=:id", {"id" : id})
    results = cursor.fetchall() # Помещаем результаты запроса в переменную
    cursor.close() # Закрываем курсор
    
    if len(results) == 1 :
        return {"id" : results[0]["id"], "name" : results[0]["name"], "price" : results[0]["price"], "calories" : results[0]["calories"]}
    else:
        return None 
    
def foodAdd(name, price, calories):
    cursor = conn.cursor()  # Создаем курсор
    cursor.execute("INSERT INTO foods(name, price, calories) VALUES(:name, :price, :calories)", {"name" : name, "price" : price, "calories" : calories})
    conn.commit() # Сохраняем измененения в базе
    newid = cursor.lastrowid
    cursor.close() # Закрываем курсор
    return newid    

def foodUpdate(id, name, price, calories):
    cursor = conn.cursor()  # Создаем курсор
    cursor.execute("UPDATE foods SET name=:name, price=:price, calories=:calories WHERE id=:id", {"id" : id, "name" : name, "price" : price, "calories" : calories})
    conn.commit() # Сохраняем измененения в базе
    cursor.close() # Закрываем курсор
    
def foodDel(id):
    cursor = conn.cursor()  # Создаем курсор
    cursor.execute("UPDATE foods SET deleted=1 WHERE id=:id", {"id" : id})
    conn.commit() # Сохраняем измененения в базе
    cursor.close() # Закрываем курсор                                                           
    
def clearDB():
    cursor = conn.cursor()  # Создаем курсор
    cursor.execute("DELETE FROM foods")
    conn.commit() # Сохраняем измененения в базе
    cursor.close() # Закрываем курсор

In [ ]:
# GET /test

# Остновная страница сервиса, просто чтобы проверить его доступность
# У этой страницы нет параметров

print('Hello!') 

In [ ]:
# GET /food

# Возвращает список блюд
# У этой страницы нет параметров

print(json.dumps(foodGetList())) # Возвращаем данные в формате JSON



In [ ]:
# GET /food/:id

# Возвращает информацию по блюду
# У этой страницы параметр передается в адресной строке

request = json.loads(REQUEST) # Получаем полный список переданных микросервису параметров
path = request['path'] # Получаем список параметров, переданных в адресной строке

if path.get('id') != None:
    id = int(path.get('id')) # Получаем id блюда

    print(json.dumps(foodInfo(id)))  # Возвращаем данные в формате JSON

In [ ]:
# PUT /food  

# Добавляет новое блюдо
# У этой страницы только PUT-параметры

request = json.loads(REQUEST) # Получаем полный список переданных микросервису параметров
args = request['args'] # Получаем список PUT-параметров

name = None
price = None
calories = None

# Получаем PUT-параметры
if 'name' in args:
    name = args['name'][0]
if 'price' in args:
    price = int(args['price'][0])
if 'calories' in args:
    calories = int(args['calories'][0])
    
newid = foodAdd(name, price, calories)
print(json.dumps({'ok' : 1, 'newid' : newid})) # Возвращаем JSON со статусом "ок" и номером нового блюда

In [ ]:
# POST /food/:id

# Обновляет информацию по блюду
# У этой страницы и PUT-параметры и параметры в адресной строке 

request = json.loads(REQUEST) # Получаем полный список переданных микросервису параметров
path = request['path'] # Получаем список параметров, переданных в адресной строке
args = request['args'] # Получаем список PUT-параметров

if path.get('id') != None:
    id = int(path.get('id')) # Получаем id блюда

    name = None
    price = None
    calories = None

    # Получаем PUT-параметры
    if 'name' in args:
        name = args['name'][0]
    if 'price' in args:
        price = int(args['price'][0])
    if 'calories' in args:
        calories = int(args['calories'][0])

    foodUpdate(id, name, price, calories) # Редактируем блюдо
    print(json.dumps({'ok' : 1})) # Возвращаем JSON со статусом "ок"

In [ ]:
# DELETE /food/:id

# Удаляет блюдо
# У этой страницы параметры в адресной строке 

request = json.loads(REQUEST) # Получаем полный список переданных микросервису параметров

if path.get('id') != None:
    id = int(request['path'].get('id')) # Получаем id удаляемого блюда

    foodDel(id) # Удаляем блюдо
    print(json.dumps({'ok' : 1})) # Возвращаем JSON со статусом "ок"

In [ ]:
# GET /cleardb

# Полная очистка базы
# У этой страницы нет параметров

clearDB( )  
print(json.dumps({'ok' : 1})) # Возвращаем JSON со статусом "ок"